In [1]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

In [2]:
folder_path = r'C:\Users\Rafael_Fagundes\Downloads\delve_compiled_data.csv'

df = pd.read_csv(folder_path)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826516 entries, 0 to 1826515
Data columns (total 41 columns):
 #   Column                                Dtype  
---  ------                                -----  
 0   Unnamed: 0                            int64  
 1   Audience Type                         object 
 2   Tactic                                object 
 3   Media Buy Name                        object 
 4   Campaign Name                         object 
 5   Fiscal Quarter                        object 
 6   Fiscal Week Year                      object 
 7   Country                               object 
 8   Display Funnel Mapped                 object 
 9   Display Dell Vehicle Mapped           object 
 10  Display Business Unit Mapped          object 
 11  Display Site Name Mapped              object 
 12  Audience Type Name                    object 
 13  Impressions                           float64
 14  Clicks                                float64
 15  Spend          

In [4]:
# Filter the dataset by US, 2024Q3 and 3P
df_filtered = df[
                (df['Fiscal Quarter'].isin(['2024-Q3','2024-Q2']))
                 & (df['Country'] == 'United States')
                 & (df['Audience Type'] == '3PD') 
                 & (df['BU'] == 'CSB') 
                 #& (df['Display Dell Vehicle Mapped'] == 'Display')
                ]

In [5]:
# Flag the outliers

# Calculate z-scores for CTR
CTR = df_filtered['Clicks'] / df_filtered['Impressions']
z_scores = np.abs((CTR - CTR.mean()) / CTR.std())

# Set a threshold for outliers (e.g., z-score > 3)
threshold = 3
df_filtered['CTR_outlier'] = z_scores > threshold

In [6]:
# Remove Outliers

# Remove Rows where Clicks are higher than Impressions
df_filtered = df_filtered[(df_filtered['Clicks'] <= df_filtered['Impressions']) 
                          & ~df_filtered['CTR_outlier'] 
                          & (df_filtered['Impressions'] > 1000)]

In [7]:
# Create a flag for the main Vendors
df_filtered['Permutive_flag'] = df_filtered['Media Buy Name'].str.contains(r'Permutive|Permutvie|permu|Permu|ermu', case=False)

df_filtered['TTD_flag'] = df_filtered['Media Buy Name'].str.contains(r'The Trade Desk', case=False) | (df_filtered['Audience Source'].str.contains(r'The Trade Desk', case=False))

df_filtered['LR_flag'] = df_filtered['Media Buy Name'].str.contains(r'Liveramp', case=False) | (df_filtered['Audience Source'].str.contains(r'Liveramp', case=False))

#df_filtered['NS_flag'] = df_filtered['Media Buy Name'].str.contains(r'Neustar', case=False) | (df_filtered['Audience Source'].str.contains(r'Neustar', case=False))

df_filtered['DV360_flag'] = df_filtered['Media Buy Name'].str.contains(r'dv360', case=False) | (df_filtered['Audience Source'].str.contains(r'dv360', case=False))

In [8]:
# Create a column with all provider names
df_filtered['Provider'] = np.where(df_filtered['Permutive_flag'], 'Permutive', 
                                   np.where(df_filtered['TTD_flag'], 'TTD', 
                                            np.where(df_filtered['LR_flag'], 'LiveRamp', 
                                                     np.where(df_filtered['DV360_flag'] & ~df_filtered['Permutive_flag'] & ~df_filtered['LR_flag'], 'DV360', None))))

In [9]:
# Create Column with DV360 + Audicen Type Name
df_filtered['Provider_lvl2'] = df_filtered.apply(lambda row: 'DV360 - ' + row['Audience Type Name'] if row['Provider'] == 'DV360' else row['Provider'], axis=1)

In [10]:
# Group KPIs by Provider
grouped_df = df_filtered.groupby([
    'Fiscal Week Year',
    'Provider_lvl2'
]).agg({
    'Impressions': 'sum',
    'Clicks': 'sum',
    'Spend': 'sum',
    'Net Rev': 'sum',
    'HVE': 'sum'
}).reset_index()

In [11]:
# Create additional KPIs
grouped_df['CTR'] = (grouped_df['Clicks'] / grouped_df['Impressions']) * 100
grouped_df['CPM'] = 1000 * grouped_df['Spend'] / grouped_df['Impressions']
grouped_df['CPC'] = grouped_df['Spend'] / grouped_df['Clicks']

grouped_df.head()

,Fiscal Week Year,Provider_lvl2,Impressions,Clicks,Spend,Net Rev,HVE,CTR,CPM,CPC


In [12]:
plt.figure(figsize=(8, 6))
ax = sns.boxplot(data=grouped_df, 
                 x='Provider_lvl2', 
                 y='CTR', 
                 showfliers=False, 
                 order=sorted(grouped_df['Provider_lvl2'].unique()))

# Rotate x-axis labels
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

# Show the plot
plt.show()

ValueError: min() arg is an empty sequence

<Figure size 800x600 with 0 Axes>

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.boxplot(data=grouped_df, 
                 x='Provider_lvl2', 
                 y='CPM', 
                 showfliers=False, 
                 order=sorted(grouped_df['Provider_lvl2'].unique()))

# Rotate x-axis labels
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

# Show the plot
plt.show()

In [ ]:
# Pie Chart data
pie_df = grouped_df.groupby(['Provider_lvl2'])['Impressions'].sum().reset_index()

# Create treemap
fig = px.treemap(pie_df, path=['Provider_lvl2'],
                 values='Impressions', 
                 title='Impression Share by Provider', 
                 color_discrete_sequence=px.colors.qualitative.Set2,
                 custom_data=['Impressions'])

# Show the plot
fig.show()

In [ ]:
# Create a horizontal stacked bar chart using Plotly Express
fig = px.bar(pie_df, y='Provider_lvl2', x='Impressions',
             orientation='h', barmode='overlay', title='Horizontal Stacked Bar Chart')

# Show the plot
fig.show()


In [ ]:
df_filtered.to_csv('csv/test.csv')